#### **1. 导入模块**

导入 Python 标准库和本项目自定义库

In [1]:
# 标准库
import os
import sys

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
from src.utils import load_multiver_corpus, save_results
from src.annotator.syntax_annotation import SyntaxAnnotator, annotate_data

#### **2. 读取语料**

读取包含人工译文和大模型译文的多版本平行语料库

In [2]:
# 指定语料库文件路径 data/output/1_mt_generation.jsonl
# 文件为 JSON 格式
# 包含《鹿鼎记》原文、闵福德译文以及大模型生成译文

data_file = '../data/output/1_mt_generation.jsonl'
print(f"从 {os.path.abspath(data_file)} 文件读取数据 ...")

# 读取数据
data = load_multiver_corpus(data_file)
print(f"成功读取 {len(data)} 条平行句对")

# 预览数据：
# 导入后的数据以 DataFrame 格式存储
# 第一列：原文
# 第二列：多版本译文：
# human：人类译本（闵福德译）
# deepseek-v3.2：deepseek 译本
# qwen3-max：qwen 译本

print("数据前 5 行如下：")
data.head()

从 C:\llm_corpus_annotation_v3\data\output\1_mt_generation.jsonl 文件读取数据 ...
成功读取 300 条平行句对
数据前 5 行如下：


,id,source,targets
0,1,{'raw_text': '江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒...,{'human': {'raw_text': 'Along a coastal road s...
1,2,{'raw_text': '前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者...,{'human': {'raw_text': 'In each of the first t...
2,3,{'raw_text': '后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。'},{'human': {'raw_text': 'The four rear carts we...
3,4,{'raw_text': '女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。'},{'human': {'raw_text': 'The little girl was cr...
4,5,{'raw_text': '囚车旁一清兵恼了，伸腿在车上踢了一脚，喝道：“再哭，再哭！'},{'human': {'raw_text': 'One of the soldiers ma...


#### **3. 加载模型**

加载词性/句法标注模型

In [3]:
# 加载 HanLP 英文模型
# hanlp.pretrained.mtl.EN_TOK_LEM_POS_NER_SRL_UDEP_SDP_CON_MODERNBERT_BASE
# 详见：https://hanlp.hankcs.com/docs/api/hanlp/pretrained/mtl.html
# 首次加载需从网络下载模型至本地
# 模型大小：620 MB 

print("加载 HanLP 英文模型 ...")
annotator = SyntaxAnnotator(lang='English', device=None)
print(f"成功加载英文模型：{annotator.model_name}")

加载 HanLP 英文模型 ...


成功加载英文模型：ModernBERT Base


#### **4. 批量标注**

使用 HanLP 标注英文各译本的词性和依存关系

并以 JSON 格式返回标注结果

In [4]:
# 注意：
# 本地大模型在 GPU 上的运行速度远快于 CPU
# HanLP 自动用户配置，自动选取 CPU 或 GPU

# 若电脑未配备 GPU，建议注册魔塔社区：https://modelscope.cn/
# 启动云端 GPU 环境运行本程序

# 开始标注
results = annotate_data(data, annotator)

Syntax Tagging: 100%|████████████████████████████████████████████████████████████████| 300/300 [00:32<00:00,  9.30it/s]


#### **5. 保存结果**

将 HanLP 标注结果保存于 JSON 文件

In [5]:
# 指定输出文件
out_file = f"../data/output/2_syntax_annotation.jsonl"

# 保存标注结果
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/2_syntax_annotation.jsonl



#### **6. 查看结果**

预览英文标注结果，查看词性/句法赋码集

In [6]:
# 预览前 3 条数据
for res in results[:3]:
    # 提取数据
    source = res['source']
    targets = res['targets']

    print(f"\n[ID]: {res['id']}")

    # 中文
    # 打印原文
    src_text = source['raw_text']
    print(f"[Source] {src_text}")
    print("-" * 60)

    # 英文
    for version, anno in targets.items():
        if not anno: continue
        print(f"[{version.upper()}]")
        
        # 打印各版本首句句法特征
        tgt_text =anno['raw_text']
        tokens = anno['tokens'][0]
        lemma = anno['lem'][0]
        pos = anno['pos'][0]
        dep = anno['dep'][0]
        
        print(f"Text: {tgt_text}")
        print(f"TOK: {tokens[:5]} ...")
        print(f"LEM: {lemma[:5]} ...")
        print(f"POS: {pos[:5]} ...")
        print(f"DEP: {dep[:5]} ...")
        
        print("=" * 60)


[ID]: 000000
[Source] 江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
------------------------------------------------------------
[HUMAN]
Text: Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
TOK: ['Along', 'a', 'coastal', 'road', 'somewhere'] ...
LEM: ['along', 'a', 'coastal', 'road', 'somewhere'] ...
POS: ['IN', 'DT', 'JJ', 'NN', 'RB'] ...
DEP: [(4, 'case'), (4, 'det'), (4, 'amod'), (26, 'obl'), (4, 'advmod')] ...
[DEEPSEEK-V3.2]
Text: Along a broad road near the coast south of the Yangtze River, a troop of Qing soldiers, armed with swords and spears, escorted seven prison carts northward, braving the biting wind and cold.
TOK: ['Along', 'a', 'broad', 'road', 'near'] ...
LEM: ['along', 'a', 'broad', 'road', 'near'] ...
POS: ['IN', 'DT', 'JJ', 'NN', 'IN'] ...
DEP: [(4, 'case'), (4, 'det'), (4, 'amod'), (26, 'obl'), (7, 'case')]